# PROYECTO REDES NEURONALES CON TENSORFLOW 

Este proyecto busca dar una idea del 'cómo se hace' y 'por qué de los resultados' de un modelo simple de clasifiación de red neuronal convolucional (CNN)  

## Requerimientos

Este proyecto es hecho en python, requiere ciertas librerías para que funcione correctamente, es por eso  que se requiere su instalación previa, lo primero es que se está usando una versión de python 3.11 (usa ```python --version``` para verificar tu versión de python en el entorno virtual), y se usan las siguientes librerías: 
 

<details>
  <summary>Tensorflow</summary>

```shell
pip install tensorflow tensorflow-datasets
pip install tensorflow[and-cuda] #si se tiene tarjeta de video para uso de core CUDA
```

</details>

<details>
  <summary>OS</summary>

Ya viene con python, no requiere instalación. Si requiere instalación use: 
```shell
pip install os
```

</details>

<details>
  <summary>pandas</summary>

```shell
pip install pandas
```

</details>


<details>
  <summary>Scikit-learn | sklearn</summary>

```shell
pip install scikit-learn
```

</details>


In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

## Configuración 
Aquí importan donde está guardados los assets del proyecto, también es importante tener en cuenta donde se encuentra el entorno virtual ese es tu raiz. 
<!-- TOC -->
- [IMG_DIR] es ruta del directorio donde se encuentran las imagenes
- [ATTR_FILE] es la ruta de los atributos normalizados y organizados, documento list_attr_celeba.csv 
- [IMG_SIZE] Es el tamaño de la imagen a trabajar, el tamaño de las imagenes es de 178x218 y se ajustan a 128x128
- [BATCH_SIZE] es el tamaño de los lotes de datos, grupitos de 32 imagenes 
- [SELECTED_ATTRS] se seleccionan 3 atributos de los normalizados en el documento list_attr_celeba.csv, ``` 'Smiling', 'Male', 'Young' ```, de los posibles 40 atributos que tiene el documento
<!-- /TOC -->

In [ ]:
IMG_DIR = "ProyectoFinal/img_align_celeba"
ATTR_FILE = "ProyectoFinal/list_attr_celeba.csv"
IMG_SIZE = (128, 128) 
BATCH_SIZE = 32 
SELECTED_ATTRS = ['Smiling', 'Male', 'Young']

## Carga y preparación de los datos 
 
 Uso de la libería pandas para definir el dataframe seleccionando las columnas de interés, es importante notar que el set de list_attr_celeba.csv usa binario de manera (-1, 1), y nostros vamos a usar (0, 1) donde 0 es -1, se hace reemplazo también aquí


In [ ]:
df = pd.read_csv(ATTR_FILE)
df.set_index('image_id', inplace=True)
df = df[SELECTED_ATTRS]
df = df.replace(-1, 0)

## División del data set

Ahora se tiene hacer la división de los datos, para el ejemplo se usa un 80% para el entrenamiento y un 20% para las pruebas; random_state=42 es para que se pueda repetir el experimento, divide siempre igual, si se desea que cada vez sea diferente se elimina esta parte (seed). 

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42) 

## Carga de las imagenes 

Función de carga de imagenes, carga una imagen desde el disco y la convierte en un array (valores 0 y 1), también devuelve la etiqueta correspondiente 
``` image_id_str = image_id.numpy().decode('utf-8')   ``` convierte de tensor que es un array de bytes a string

In [ ]:
def load_image_and_labels(image_id, label):
    image_id_str = image_id.numpy().decode('utf-8')  
    img_path = os.path.join(IMG_DIR, image_id_str)
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    return img_array, label

## Conversión tf.data.Dataset 

Convierte el dataframe en un objeto tipo ``` tf.data.Dataset ```, este es un tipo de objeto que usa tensorflow y representa una secuencia de elementos ( ejemplos de entrenamiento ) Cada elemento puede ser un para (x,y) (dato, etiqueta) o cualquier estructura de datos. 

Sirve para crear piplines eficientes de entrada de datos ( lectura, transformación, batching ...) es crucial para entrenar modelos grandes.

### ¿Por qué se usa tf.data.Dataset? 
<!-- TOC -->

-[Esalable] Maneja grandes volúmenes de datos que no caben en memoria
-[Eficiente] Usa pipelines paralelos, caching, prefetching (técnicas de optimización en ``` tf.data.Dataset ```)
-[Flexible] se puede hacer transformaiciones complejas y ser leidos de archivos
-[Integración_directa_con-model.fit()] los modelos Keras aceptan Dataset como entrada

<!-- /TOC -->

Usa ``` tf.py_function ``` para integrar funciones de python puras dentro del pipline de TensorFlow

Al final de la función se hace la conversión del los dataframe

In [ ]:
def df_to_dataset(dataframe):
    image_ids = dataframe.index.values
    labels = dataframe.values.astype('float32')
    dataset = tf.data.Dataset.from_tensor_slices((image_ids, labels))

    def process(x, y):
        img, label = tf.py_function(load_image_and_labels, [x, y], [tf.float32, tf.float32])
        img.set_shape((128, 128, 3))  # Establecer forma explícita
        label.set_shape((len(SELECTED_ATTRS),))
        return img, label

    dataset = dataset.map(process, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset


train_ds = df_to_dataset(train_df)
val_ds = df_to_dataset(val_df)

## Definición del modelo

Aquí se define una red neuronal convolucional (CNN) con 3 capas convolucionales y una capa densa final con activación sigmoid para clasificación multietiquieta

### Redes neuronales convolucionales (CNN)

Se usan sobre todo para análisis de imágenes

- Las capas convolucionales (``` Conv2D ```) detectan patronas visuales (bordes, texturas ...)
- Las capas de pooling ( ``` MaxPooling2D ```) reducen el tamaño y conservan lo importante
- Las capas densas ( ``` Dense ```) conectan todos los nodos y toman decisiones (clasificación)


### ¿Qué es keras? 

Keras es una API de alto nivel para construir y entrenar redes neuronales, desde la versión 2.x de tensorflow keras está integrada como tf.keras

ventajas 
- Fácil de usar y leer
- Permite construir modelos rápidamente ( Sequential, Functional, Subclasssing ) 
- Se integra directamente con el entrenamiento, evaluación y exportación de modelos

### Sigmoid

la función sigmoid  $$σ(x) = \frac{1}{1 + e^{-x}}$$, donde x es el valor de entrada ( puede venir de una neurona o de una capa densa )

- toma cualquier número real y lo aplana a un valor entre 0 y 1
- Se interpreta como una probabilidad 

En la clasificación multietiqueta cada ejemplo puede tener más de una clase verdadera al mismo tiempo ( para nuestro caso puede ser un hombre joven sonriendo ``` 'Smiling', 'Male', 'Young' ``` ) 
- cada neurona de salida actúa de forma independiente, indicando la probabilidad de que esa etiqueta esté presente

si hay 5 etiquetas ``` [ 0.02, 0.97, 0.22, 0.01, 0.86 ] ``` se puede interpretar como que las etiquetas 2 y 5 están presentes (verdadera)

### Softmax 

 - [sigmoid] se usa par multietiqueta, cada salida es independiente 
 - [softmax] se usa para multiclase, donde solo una etiqueta es verdadera y las probabilidades suman 1 


 ``` tf.keras.layers.Dense(len(SELECTED_ATTRS), activation='sigmoid') ``` Una neurona por etiqueta y devuelve un valor entre 0 y 1 (sigmoid) indicando la probabilidad de que esa etiqeuta esté presente en la imagen 



In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(SELECTED_ATTRS), activation='sigmoid')
])

## Compilación y entrenamiento

- Con el optimizador Adam y la función de pérdida binary_crossentropy se hace la compilación 
- se entrena durante  5 épocas 

``` model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) ``` método para preparar el modelo apra el entrenamiento, se requiere definir: 
- Un optimizador ( ``adam`` )
- Una función de pérdida ( ``binary_crossentropy`` )
- Mérticas para seguimiento ( ``accuracy`` )

### Optimizador Adam 

El optimizador ajusta los pesos del modelo en cada paso del entrenamiento para que la función pédida disminuya, el llamado `` motor de aprendizaje ``
Adam (  Adaptive Moment Estimation ) es un algoritmo de optimización avanzado, combina lo mejor de otros algoritmos: 
- Momentum usa el promedio de gradientes pasados
- RMSProp adapta la tasa de aprendizaje a cada parámetro 

de los optimizadores mas usados por eficacia y facilidad de uso 

ventajas 
- Rápido y preciso 
- Pcoas cosas que se deben ajustaar ( se puede usar con valores por defecto )
- Ideal para grandes datasets y redes profundas 

### Función de pérdida binary_crossentropy 

Se usa para clasificaciones multietiqueta o binarios, En multietiqueta cálcula una pérdidad por cada etiqueta y luego se hace un promedio 


### Entrenamiento 

El entrenamiento es el proceso donde el modelo: 

1. Toma un lote de datos (batch) 
2. Hace predicciones 
3. Calcula el error usando la función de pérdida ( ``binary_crossentropy`` )
4. Ajusta los pesos usando el optimizador (`` Adam `` )


- Se imprime la pérdida ( ` loss ` ) y la exactitud ( `accuracy ` ) del entrenamiento y validación
- Si la pérdida baja, el modelo está aprendiendo 
- Puede detener el entrenamiento cuando estés satisfecho o usando técnicas como el ` early stopping `


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento
model.fit(train_ds, validation_data=val_ds, epochs=5)